In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_digits, fetch_openml
from sklearn.model_selection import train_test_split

from builtins import range
from csv import reader
from random import randrange
import numpy as np
import pandas as pd 
import csv
import matplotlib.pyplot as plt
import random
import math
random.seed(113)
import warnings
from sklearn.metrics import classification_report
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
input_data.read_data_sets('C:/Users/mina/Desktop')

from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

with open('C:/Users/mina/Desktop/train-images-idx3-ubyte.gz', 'rb') as f:
  train_images = extract_images(f)
with open('C:/Users/mina/Desktop/train-labels-idx1-ubyte.gz', 'rb') as f:
  train_labels = extract_labels(f)

with open('C:/Users/mina/Desktop/t10k-images-idx3-ubyte.gz', 'rb') as f:
  test_images = extract_images(f)
with open('C:/Users/mina/Desktop/t10k-labels-idx1-ubyte.gz', 'rb') as f:
  test_labels = extract_labels(f)

  train_labels.shape = (60000,1)
  test_labels.shape = (10000,1)
  
#train_labels=train_labels.transpose()
#test_labels=test_labels.transpose()

print(train_labels.shape)
print(test_labels.shape)

import numpy as np
import keras

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

no_of_different_labels =10

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)


train_images = np.concatenate((train_labels, train_images),axis=1)
test_images = np.concatenate((test_labels, test_images),axis=1)

print(train_images.shape) # (60000, 785)
print(test_images.shape)  # (10000, 785)
train_images

fac = 0.99 / 255
train_imgs = np.asfarray(train_images[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_images[:, 1:]) * fac + 0.01
train_labels = np.asfarray(train_images[:, :1])
test_labels = np.asfarray(test_images[:, :1])

pip install keras tensorflow numpy mnist

import numpy as np
lr = np.arange(10)
for label in range(10):
    one_hot = (lr==label).astype(np.int)
    print("label: ", label, " in one-hot representation: ", one_hot)

import pandas as pd
import numpy as np
lr = np.arange(no_of_different_labels)
# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(np.float)
test_labels_one_hot = (lr==test_labels).astype(np.float)
# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99


for i in range(10):
    img = train_imgs[i].reshape((28,28))
    plt.imshow(img, cmap="Greys")
    plt.show()

import numpy as np
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, 
                     (upp - mean) / sd, 
                     loc=mean, 
                     scale=sd)
class NeuralNetwork:
    
    def __init__(self, 
                 no_of_in_nodes, 
                 no_of_out_nodes, 
                 no_of_hidden_nodes,
                 learning_rate):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
        
    def create_weight_matrices(self):
        """ A method to initialize the weight matrices of the neural network"""
        rad = 1 / np.sqrt(self.no_of_in_nodes)
        X = truncated_normal(mean=0, 
                             sd=1, 
                             low=-rad, 
                             upp=rad)
        self.wih = X.rvs((self.no_of_hidden_nodes, 
                                       self.no_of_in_nodes))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes)
        X = truncated_normal(mean=0, 
                             sd=1, 
                             low=-rad, 
                             upp=rad)
        self.who = X.rvs((self.no_of_out_nodes, 
                                        self.no_of_hidden_nodes))
        
    
    def train_single(self, input_vector, target_vector):
        """
        input_vector and target_vector can be tuple, 
        list or ndarray
        """
        
        output_vectors = []
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.wih, 
                                input_vector)
        output_hidden = activation_function(output_vector1)
        
        output_vector2 = np.dot(self.who, 
                                output_hidden)
        output_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_network
        # update the weights:
        tmp = output_errors * output_network * \
              (1.0 - output_network)     
        tmp = self.learning_rate  * np.dot(tmp, 
                                           output_hidden.T)
        self.who += tmp
        
        who_old = 0
        wih_old = 0
        if(len(self.intermediate_weights) > 0):
            who_old = self.intermediate_weights[-1][1]
            wih_old = self.intermediate_weights[-1][0]
        
        # Adding the momentum value for who update
        self.who += who_old * 0.5
        
        # calculate hidden errors:
        hidden_errors = np.dot(self.who.T, 
                               output_errors)
        # update the weights:
        tmp = hidden_errors * output_hidden * (1.0 - output_hidden)
        self.wih += self.learning_rate * np.dot(tmp, input_vector.T)
        
        # Adding the momentum value for wih update
        self.wih += wih_old * 0.5
        
    def train(self, data_array, 
              labels_one_hot_array,
              epochs=1,
              intermediate_results=False):
        
        for epoch in range(epochs):
            self.intermediate_weights = []
            print("*", end="")
            for i in range(len(data_array)):
                self.train_single(data_array[i], 
                                  labels_one_hot_array[i])
            if intermediate_results:
                self.intermediate_weights.append((self.wih.copy(), 
                                             self.who.copy()))
        return self.intermediate_weights        
            

        
    def run(self, input_vector):
        # input_vector can be tuple, list or ndarray
        input_vector = np.array(input_vector, ndmin=2).T
        output_vector = np.dot(self.wih, 
                               input_vector)
        output_vector = activation_function(output_vector)
        
        output_vector = np.dot(self.who, 
                               output_vector)
        output_vector = activation_function(output_vector)
    
        return output_vector
            
    def confusion_matrix(self, data_array, labels):
        cm = np.zeros((10, 10), int)
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            cm[res_max, int(target)] += 1
        return cm    
    def precision(self, label, confusion_matrix):
        col = confusion_matrix[:, label]
        return confusion_matrix[label, label] / col.sum()
    
    def recall(self, label, confusion_matrix):
        row = confusion_matrix[label, :]
        return confusion_matrix[label, label] / row.sum()
        
    
    def evaluate(self, data, labels):
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i]:
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs
            

epochs = 10
ANN = NeuralNetwork(no_of_in_nodes = 784, 
                               no_of_out_nodes = 10, 
                               no_of_hidden_nodes = 50,
                               learning_rate = 0.1)
    
    
 
weights = ANN.train(train_imgs, 
                    train_labels_one_hot, 
                    epochs=epochs, 
                    intermediate_results=True)

cm = ANN.confusion_matrix(train_imgs, train_labels)
        
print(ANN.run(train_imgs[i]))

print(cm)
corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
print("accruracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
print("accruracy: test", corrects / ( corrects + wrongs))
cm = ANN.confusion_matrix(train_imgs, train_labels)
print(cm)
for i in range(10):
    print("digit: ", i, "precision: ", ANN.precision(i, cm), "recall: ", ANN.recall(i, cm))

#from sklearn.metrics import classification_report, confusion_matrix
#print(confusion_matrix(train_imgs, test_imgs))

for i in range(epochs):  
    print("epoch: ", i)
    ANN.wih = weights[i][0]
    ANN.who = weights[i][1]
   
    corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
    print("accruracy train: ", corrects / ( corrects + wrongs))
    corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
    print("accruracy test:", corrects / ( corrects + wrongs))